General Overview:
1. Understand the shape of the data
2. Data Cleaning
3. Data Exploration
4. Feature Enginerring
5. Data Preprocessing for Model
6. Basic Model Buiilding
7. Model Tuning
8. Ensemble Model Building
9. Results

### 1. Understand the shape of the data

In [2]:
import numpy as np
import pandas as pd
import re
import calendar

import seaborn as sns
import matplotlib.style as style


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# grid: rgb(240,240,240)
# background: rgb(200,200,200)
style.use('fivethirtyeight')

In [12]:
#import required packages
#basics
import pandas as pd 
import numpy as np

#misc
import gc
import time
import warnings

#stats
# from scipy.misc import imread
from scipy import sparse
import scipy.stats as ss

#viz
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
import matplotlib_venn as venn

#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer   

In [14]:
train = pd.read_csv('../data/train.csv')
def getting_to_know(df, question=None):
    display(df.shape)
    display(df.columns)
    if question == 'y':
        display(df.head())
        display(df.tail())
    print('--------------------------END--------------------------------')
getting_to_know(train, 'y')

(159571, 8)

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


--------------------------END--------------------------------


Let's create two dataframes containing only and no toxic comments, respectively:

In [22]:
no_toxic_train = train[(train['toxic'] == 0)
   & (train['severe_toxic'] == 0)
   & (train['obscene'] == 0) 
   & (train['threat'] == 0) 
   & (train['insult'] == 0) 
   & (train['identity_hate'] == 0)]

toxic_train = train[(train['toxic'] == 1)
   | (train['severe_toxic'] == 1)
   | (train['obscene'] == 1) 
   | (train['threat'] == 1) 
   | (train['insult'] == 1) 
   | (train['identity_hate'] == 1)]

In [23]:
train.shape[0] == (no_toxic_train.shape[0] + toxic_train.shape[0])

True

Let's see some values

In [24]:
train['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [25]:
pd.crosstab(train.toxic, [train.obscene, train.threat, train.insult, train.identity_hate])

obscene             0                                    1                     \
threat              0                    1               0                  1   
insult              0          1         0      1        0         1        0   
identity_hate       0    1     0    1    0  1   0  1     0   1     0    1   0   
toxic                                                                           
0              143346   54   301   28   22  0   3  0   317   3   181   18   2   
1                5707  139  1229  141  124  8  17  3  1916  41  4789  883  15   

obscene                 
threat                  
insult           1      
identity_hate    0   1  
toxic                   
0                2   0  
1              195  87

In [26]:
pd.crosstab(train.severe_toxic, [train.obscene, train.threat, train.insult, train.identity_hate])

obscene             0                                    1                     \
threat              0                    1               0                  1   
insult              0          1         0      1        0         1        0   
identity_hate       0    1     0    1    0  1   0  1     0   1     0    1   0   
severe_toxic                                                                    
0              149012  190  1516  162  135  7  19  3  2075  38  3981  636  13   
1                  41    3    14    7   11  1   1  0   158   6   989  265   4   

obscene                 
threat                  
insult           1      
identity_hate    0   1  
severe_toxic            
0              133  56  
1               64  31